In [1]:
import pandas as pd

#Importo el archivo de GAMES
games=pd.read_csv("Archivos_Comprimidos/games.csv",encoding="latin-1")

#Importo el archivo de REVIEWS
reviews= pd.read_parquet("Archivos_Comprimidos/reviews_final.parquet")

#Importo el archivo de ITEMS
items=pd.read_parquet("Archivos_Comprimidos/items.parquet")

#DataFrame joineado para funciones 1) y 2). Voy a unir games con items

In [2]:
# Games:
games.head(2)

,genres,app_name,date,price,item_id
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018,4.99,761140.0
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018,Free To Play,643980.0


In [3]:
# Items:
items.head(2)

,user_id,item_id,item_name,horas
0,76561197970982479,10,Counter-Strike,0.1
1,76561197970982479,20,Team Fortress Classic,0.0


In [4]:
# Reviews
reviews.head(2)

,user_id,posted,item_id,review
0,76561197970982479,2011.0,1250,Simple yet with great replayability. In my opi...
1,76561197970982479,2011.0,22200,It's unique and worth a playthrough.


In [5]:
#Join los dfs: "items" con "games"
#En la tabla items tengo la cantidad de horas jugadas para cada usuario por cada juego (item)
#Asocio a esta tabla el genero del juego
#Las uno mediante item_id 

items['item_id'] = items['item_id'].astype(float) #Necesito igualar el tipo de dato para poder joinear las tablas 
df= pd.merge(items, games, on='item_id', how='inner')
df.tail(3)   #df que voy a utlizar en la funcion

,user_id,item_id,item_name,horas,genres,app_name,date,price
4142660,76561198081168593,375450.0,NOBUNAGA'S AMBITION: Souzou SengokuRisshiden,0.850000,['Strategy'],NOBUNAGA'S AMBITION: Sphere of Influence - Asc...,2016,59.99
4142661,76561198107283457,354280.0,ChaosTower,2.733333,"['Action', 'Casual', 'Indie', 'Simulation']",ChaosTower,2016,5.99
4142662,inven,433920.0,Aveyond 4: Shadow Of The Mist,0.000000,"['Adventure', 'Indie', 'RPG']",Aveyond 4: Shadow of the Mist,2016,14.99


In [6]:
#FUNCION 1:  PlayTimeGenre
# 1) Paso toda al columna de generos a minuscula 
df['genres'] = df['genres'].str.lower()
#df['genres'].tail(5)

# 2) Filtro los valores que contienen el genero que me interesa
genero="casual"
df_filtrado = df[df.filter(like='genres').apply(lambda x: x.str.contains(genero)).any(axis=1)] 
#df_filtrado['genres'].tail(5)
##Me devuelve los valores booleanos True que cumplen la condicion del filtro.


#Ya seleccione los datos  que necesito. Me falta agruparlos y sumar las horas de juego por año.
#3) Agrupo los valores por año en cada caso.

df_ordenado=df_filtrado[["date","horas"]].groupby(["horas"]).sum().sort_values(by='horas',ascending=False)

#df_ordenado.head(10)

df_ordenado.reset_index(inplace=True) #Agrego el indice
df_ordenado.head(10)


#df_filtrado["date"].value_counts()

#df_ordenado['horas'] = df_ordenado['horas'].astype(int)

df_ordenado.head(100)

#print(int(df_ordenado["horas"].iloc[0]))
#print(df_ordenado["date"].iloc[0])


print(f"Año de lanzamiento con más horas jugadas para género: {genero} --> {df_ordenado.date.iloc[0]}")
print(f"Cantidad de horas jugadas en dicho año: {int(df_ordenado.horas.iloc[0])}")

Año de lanzamiento con más horas jugadas para género: casual --> 2012
Cantidad de horas jugadas en dicho año: 10588


In [7]:
#Resumen de Funcion 1: 

# 1) Paso toda al columna de generos a minuscula 
df['genres'] = df['genres'].str.lower()

# 2) Filtro los valores que contienen el genero que me interesa
genero="casual" # a modo prueba
genero=genero.lower()
df_filtrado = df[df.filter(like='genres').apply(lambda x: x.str.contains(genero)).any(axis=1)] 

#3) Agrupo los valores por año.
df_ordenado=df_filtrado[["date","horas"]].groupby(["horas"]).sum().sort_values(by='horas',ascending=False)

#4) Reseteo indice numerico para ver las posiciones
df_ordenado.reset_index(inplace=True)
df_ordenado.head(100)

# 4) Imprimo resultado de funcion
print(f"Año de lanzamiento con más horas jugadas para género {genero.upper()}: {df_ordenado.date.iloc[0]}")
print(f"Cantidad de horas jugadas en dicho año: {int(df_ordenado.horas.iloc[0])}")

Año de lanzamiento con más horas jugadas para género CASUAL: 2012
Cantidad de horas jugadas en dicho año: 10588


In [8]:
#Primera Funcion

def PlayTimeGenre( genero : str ):
    genero=genero.lower() #lo paso a minusculas por si esta escrito con mayusculas
    # 1) Paso toda al columna de generos a minuscula
    df['genres'] = df['genres'].str.lower()
    
    # 2) Filtro los valores que contienen el genero que me interesa
    #genero="action" # a modo prueba
    df_filtrado = df[df.filter(like='genres').apply(lambda x: x.str.contains(genero)).any(axis=1)] 
    
    #3) Agrupo los valores por año.
    df_ordenado=df_filtrado[["date","horas"]].groupby(["horas"]).sum().sort_values(by='horas',ascending=False)
    
    #4) Reseteo con indices numericos
    df_ordenado.reset_index(inplace=True) #Agrego el indice para ver las posiciones
    
    #5) Imprimo resultado de funcion
    print(f"Cantidad de horas jugadas en dicho año: {int(df_ordenado.horas.iloc[0])}")
    return 


PlayTimeGenre("strategy")

Cantidad de horas jugadas en dicho año: 8848


In [9]:
#FUNCION 2:  UserForGenre
# 1) Paso toda la columna de generos a minuscula 
df['genres'] = df['genres'].str.lower()
genero=genero.lower() #paso a minuscula en caso de que no este de la misma forma

# 2) Filtro los valores que contienen el genero que me interesa
#genero="action"
df_filtrado = df[df.filter(like='genres').apply(lambda x: x.str.contains(genero)).any(axis=1)] 
df_filtrado['genres'].tail(5)   #Me devuelve los valores booleanos True que cumplen la condicion del filtro.

#3) Creo un nuevo df para ver la cant de horas por jugador/userid
df_ordenado=df_filtrado[["user_id","horas"]].groupby(["horas"]).sum().sort_values(by='horas',ascending=False)
#df_ordenado.head(5)

# 4)Como ordene de Mayor a Menor, obtengo el primer valor de user_id que corresponde al que mas horas jugo
userid_maxhoras=df_ordenado["user_id"].iloc[0]

print(f"Usuario con mas horas jugadas para género {genero.upper()}: {userid_maxhoras}")

# 5)Renombro la columna que contiene el anio, para evitar confusiones
df_filtrado.rename(columns={'date': 'year'}, inplace=True)   
#df_filtrado.head(5)

# 6)Ya conozco el user id, creo una mascara para quedarme solamente con los datos del user id max horas
df_filtrado = df_filtrado[df_filtrado["user_id"]==userid_maxhoras]
#df_filtrado.head(5)

#5) Me falta mostrar los valores por anio
horas_por_anio = df_filtrado.groupby('year')['horas'].sum()

#6) Creo un dataframe con los valores de la serie que cree
df_horas_por_anio = pd.DataFrame({'Año': horas_por_anio.index, 'Horas': horas_por_anio.values})

#7) Recorro el diccionario para 
for indice, fila in df_horas_por_anio.iterrows():
  año = int(fila['Año'])
  horas_jugadas = int(fila['Horas'])
  print(f"El Año: {año} jugó {horas_jugadas} horas ")

Usuario con mas horas jugadas para género CASUAL: Evilutional
El Año: 2012 jugó 10614 horas 
El Año: 2014 jugó 1 horas 
El Año: 2015 jugó 0 horas 
El Año: 2017 jugó 0 horas 


/var/folders/lh/p6j3glj52sq44ch3v_6_hltc0000gn/T/ipykernel_21911/146886013.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.rename(columns={'date': 'year'}, inplace=True)


In [10]:
#Segunda Funcion

def UserForGenre( genero : str ):
    # 1) Paso toda la columna de generos a minuscula y el parametro tambien
    df['genres'] = df['genres'].str.lower()
    genero=genero.lower()
    
    # 2) Filtro los valores que contienen el genero que me interesa
    #genero="action"
    df_filtrado = df[df.filter(like='genres').apply(lambda x: x.str.contains(genero)).any(axis=1)]
    
    #3) Creo un nuevo df para ver la cant de horas por jugador/userid
    df_ordenado=df_filtrado[["user_id","horas"]].groupby(["horas"]).sum().sort_values(by='horas',ascending=False)
    
    # 4)Como ordene de Mayor a Menor, obtengo el primer valor de user_id que corresponde al que mas horas jugo
    userid_maxhoras=df_ordenado["user_id"].iloc[0]
    print(f"Usuario con mas horas jugadas para género {genero.upper()}: {userid_maxhoras}")
    
    # 5)Renombro la columna que contiene el anio, para evitar confusiones
    df_filtrado.rename(columns={'date': 'year'}, inplace=True)
    
    # 6)Ya conozco el user id, creo una mascara para quedarme solamente con los datos del user id max horas
    df_filtrado = df_filtrado[df_filtrado["user_id"]==userid_maxhoras]
    
    #5) Agrupo las horas por anio
    horas_por_anio = df_filtrado.groupby('year')['horas'].sum()
    
    #6) Creo un dataframe con los valores de la serie que cree
    df_horas_por_anio = pd.DataFrame({'Año': horas_por_anio.index, 'Horas': horas_por_anio.values})
    
    #7) Recorro el diccionario para 
    for indice, fila in df_horas_por_anio.iterrows():
        año = int(fila['Año'])
        horas_jugadas = int(fila['Horas'])
        print(f"El Año: {año} jugó {horas_jugadas} horas ")
        
    return

UserForGenre("Action")